In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

# Convolution Neural Networks 

합성곱 신경망 

이미지 인식, 음성 인식 등에 사용됨 


# 7.1 전체 구조

* Pooling Layer 추가됨 

* Fully Connected (전결합) : 인접하는 계층의 모든 뉴런과 결합

* Affine 계층 : 완전히 연결된 계층 

        Affine -> ReLU -> Affine -> ReLU -> Affine -> Softmax ->

* CNN 구조 
    
        Conv -> ReLU -> Pooling -> Conv -> ReLU -> Pooling
        -> Conv -> ReLU -> Affine -> ReLU -> Affine -> Softmax -> 
    
    Conv 합성곱 계층과 Pooling 계층이 추가 됨 

    

# 7.2 Conv 계층

## 7.2.1 Fully Connected 계층의 문제점 

* Fully Connected : 인접하는 계층의 뉴런이 모두 연결되고 출력의 수는 임의로 정할 수 있다. 

    문제점 : 데이터의 형상이 무시 된다. <br>
    예) 3차원 이미지 데이터를 평평한 1차원 데이터로 바꿔서 연산을 해야함 MNIST 데이터 셋을 사용한 사례에서 (1,28,28) 이미지를 1줄로 784 개의 데이터를 Affine 계층에 입력으로 사용


* Feature map : 합성곱 계층의 입출력 데이터 
    input feature map, output feature map
    
    
## 7.2.2 Conv 연산

합성곱 연산은 이미지 처리에서 말하는 **필터 연산** 에 해당

가로 세로 방향의 차원을 가짐 입력 (4,4) 필터는 (3,3) -> (2,2) 출력 

<img src='CNNFilter.png' width=50%>

* kernel : 필터 

* window : 합성곱의 연산은 필터의 윈도우를 일정간격으로 이동하며 입력데이터 적용 

입력과 필터에서 대응하는 원소끼리 곱한 후 그 총합을 구함 (단일곱셈 누산 Fused multiply-add FMA 라함) 그 결과를 출력의 장소에 저장 

CNN 에서의 필터 = 가중치 

편향은 항상 하나만 존재하여 필터를 적용한 모든 원소에 같은 값을 더하여 사용 


## 7.2.3 패딩

* padding : 입력 데이터 주변의 특정 값을 채움 

-- 패딩은 주로 출력 크기를 조정할 목적으로 사용, 입력 데이터의 공간적 크기를 고정한 채로 다음 계층에 전달 할수 있다. (Conv 연산을 거칠때 마다 점점 크기가 줄어듬을 방지) 


## 7.2.4 스트라이드 

* stride : 필터를 적용하는 위치의 간격, window 가 이동하는 간격 (보폭)

입력 (H,W) 필터 (FH, FW) -> 출력 (OH, OW) <br>
패딩 P, 스트라이드 S

\begin{equation*}
    OH = \frac{H + 2P + - FH}{S} + 1 \\
    OW = \frac{H + 2P + - FW}{S} + 1
\end{equation*}

정수로 나눠 떨어져야함. 값이 딱 나눠 떨어지지 않을 때엔 가장 가까운 정수로 반올림 등 (특별히 에러를 내지 않고 진행하도록 구현)

## 7.2.5 3차원 데이터의 Conv 연산 

http://cs231n.github.io/convolutional-networks/

필터도 여러가지 채널(색상) 에 맞춰서 줘야한다. 채널이 3개면 필터도 3개 

## 7.2.6 블록으로 생각하기 

3차원의 연산은 데이터와 필터를 직육면체 블록으로 생각 

채널수 C, 필터 높이 FH, 필터 너비 FW (C, FH, FW)

<img src='CNNblock.png' width=50%/>


여러개의 필터를 사용하면 4차원이 됨 -> 출력데이터가 필터의 개수만큼 만들어짐 (FN)

형상이 다른 블록의 덧셈은 브로드캐스트됨


## 7.2.7 배치 처리 

<img src ='CNNblock2.png' width=50%/>

블럭이 N 개, 필터의 개수는 그대로 (N 데이터수, FN 필터 수 , OH 높이, OW 너비) 4차원의 데이터 

N 개의 데이터에 대한 합성곱 연산이 이뤄짐 



# 7.3 풀링계층 

Pooling layer : 가로, 세로 방향의 공간을 줄이는 연산 


<img src='CNNmaxPooling.png' width=50%>

* 최대 풀링(max pooling)을 스트라이드 2로 처리 영역의 최대값을 취함 
* 평균 풀링 : 평균 풀링은 대상 영역의 평균으로 계산 

## 7.3.1 풀링계층의 특징

* 최댓값 혹은 평균을 취하는 명확한 처리하므로 학습해야 할 매개변수가 없다.
* 채널마다 독립적으로 계산하므로 채널 수가 변하지 않는다.
* 입력 데이터가 조금 변해도 풀링의 결과는 잘 변하지 않는다. (영향을 적게 받는다, 떨림 같은..)


# 7.4 합성곱/풀링 계층 구현하기

## 7.4.1 4차원배열

* 높이 28, 너비 28, 채널 1 인 이미지 데이터 10개 (10, 1, 28, 28)


In [8]:
import numpy as np
x = np.random.rand(10, 1, 28, 28)
x.shape
x[0].shape
x[0, 0][:2]

(10, 1, 28, 28)

(1, 28, 28)

array([[ 0.63464697,  0.29145329,  0.70943543,  0.07292147,  0.61769742,
         0.0262977 ,  0.63931379,  0.88161805,  0.50876108,  0.96345767,
         0.30438957,  0.47415976,  0.89437433,  0.42327603,  0.70335345,
         0.53635078,  0.83545286,  0.12923368,  0.90939233,  0.40892987,
         0.3615447 ,  0.70294223,  0.58095422,  0.47800973,  0.84101858,
         0.92739338,  0.94328139,  0.68488791],
       [ 0.09005096,  0.38282588,  0.6024875 ,  0.79704861,  0.71563762,
         0.42587486,  0.50756603,  0.49538155,  0.57475933,  0.51511513,
         0.58623892,  0.81419385,  0.89426036,  0.5061551 ,  0.18958154,
         0.10657779,  0.18733242,  0.10495775,  0.52886277,  0.217858  ,
         0.93185952,  0.19957075,  0.07862927,  0.65470483,  0.12457137,
         0.59053177,  0.01442056,  0.34501623]])

## 7.4.2 im2col 로 데이터 펼침

넘파이에서는 원소에 접근할 때 for 문을 사용하지 않는 것이 좋음

<img src = 'CNNim2col.png' width=50%>
* im2col 함수를 구현 : 입력데이터를 필터링(가중치 계산) 하기 좋게 전개하는(펼치는)함수, 4차원 데이터가 2차원 행렬로 바뀜 
    <br>입력데이터에서 필터를 적용해 3차원 블록을 한줄로 늘어 놓음 
    
* caffe 와 chainer 에서도 im2col 이용 

<img src ='CNNim2col1.png' width=50%>

필터를 세로로 1열로 전개 N 개의 필터 만큼 [데이터열 x FN] 이 됨 <br>
2차원인 출력 데이터를 4차원의 reshape 함


In [9]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns 
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [11]:
x1 = np.random.rand(1,3,7,7) # 데이터 수, 채널, 높이, 너비 (4차원데이터)
col1 = im2col(x1, 5, 5,stride=1, pad=0)
col1.shape # 9 출력 이미지의 사이즈 75/3=25 채널 3 개로 되어 있음 1채널에는 25 = 5개 씩 5개 

(9, 75)

In [12]:
x2 = np.random.rand(10,3,7,7) 
col2 = im2col(x2, 5, 5,stride=1, pad=0)
col2.shape

(90, 75)

In [14]:
x1[:, :,:5, :5]

array([[[[ 0.49151141,  0.12164675,  0.86855103,  0.22889157,  0.14627692],
         [ 0.07541529,  0.3949526 ,  0.33361449,  0.86657293,  0.4811581 ],
         [ 0.32640807,  0.39474257,  0.82698389,  0.73426756,  0.2800613 ],
         [ 0.07286197,  0.3153743 ,  0.3133537 ,  0.87511364,  0.28247084],
         [ 0.42991487,  0.09583869,  0.14669056,  0.30303999,  0.64246304]],

        [[ 0.05630697,  0.65717021,  0.79114124,  0.45824106,  0.034449  ],
         [ 0.61192191,  0.39725512,  0.64123497,  0.75642833,  0.05970315],
         [ 0.01500802,  0.33153892,  0.80708351,  0.68613629,  0.50918153],
         [ 0.03102622,  0.61155163,  0.56794188,  0.11420271,  0.15683787],
         [ 0.12565463,  0.09268063,  0.37644713,  0.55595962,  0.53431739]],

        [[ 0.84796216,  0.07615896,  0.20359564,  0.84968551,  0.69550601],
         [ 0.49095945,  0.34566338,  0.28857688,  0.38606163,  0.02631764],
         [ 0.88426045,  0.25525849,  0.790754  ,  0.69487079,  0.05823669],
        

In [13]:
col1[0, :] #  0.05630697 6번째 -> x1 의 두번쨰 [[ 0.05630697 합쳐짐

array([ 0.49151141,  0.12164675,  0.86855103,  0.22889157,  0.14627692,
        0.07541529,  0.3949526 ,  0.33361449,  0.86657293,  0.4811581 ,
        0.32640807,  0.39474257,  0.82698389,  0.73426756,  0.2800613 ,
        0.07286197,  0.3153743 ,  0.3133537 ,  0.87511364,  0.28247084,
        0.42991487,  0.09583869,  0.14669056,  0.30303999,  0.64246304,
        0.05630697,  0.65717021,  0.79114124,  0.45824106,  0.034449  ,
        0.61192191,  0.39725512,  0.64123497,  0.75642833,  0.05970315,
        0.01500802,  0.33153892,  0.80708351,  0.68613629,  0.50918153,
        0.03102622,  0.61155163,  0.56794188,  0.11420271,  0.15683787,
        0.12565463,  0.09268063,  0.37644713,  0.55595962,  0.53431739,
        0.84796216,  0.07615896,  0.20359564,  0.84968551,  0.69550601,
        0.49095945,  0.34566338,  0.28857688,  0.38606163,  0.02631764,
        0.88426045,  0.25525849,  0.790754  ,  0.69487079,  0.05823669,
        0.56016581,  0.67255286,  0.09210888,  0.43415972,  0.63

## 7.4.3 합성곱 계층 구현

- Pooling 클래스
    - forward() 메소드
        - 입력 데이터 전개
        - 행별 최댓값
        - 적절한 모향으로 성형
    - backward() 메소드
        - ReLU 노드 구현 시 사용한 max의 역전파 참고

In [ ]:

class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        # 중간 데이터（backward 시 사용）
        self.x = None   
        self.col = None
        self.col_W = None
        
        # 가중치와 편향 매개변수의 기울기
        self.dW = None
        self.db = None

    def forward(self, x):
        FN, C, FH, FW = self.W.shape  # 필터개수, 채널, 필터높이, 필터폭
        N, C, H, W = x.shape
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        ## Conv 구현 
        col = im2col(x, FH, FW, self.stride, self.pad)
        # 2차원 배열로 만듬 
        col_W = self.W.reshape(FN, -1).T
        # 두 행렬의 내적함 
        out = np.dot(col, col_W) + self.b
        
        # (N,H,W,C) -> (N, C, H, W) 로 변경함 
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.col_W = col_W

        return out

    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx


In [26]:
# numpy reshape
x = np.array(np.arange(27)).reshape(1,3,3,3)
x
x.reshape(1,-1)

array([[[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8]],

        [[ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]]])

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26]])